In [12]:
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure

In [42]:
data = pd.read_csv("output.csv")
data = data.head(10)
#data = data.transpose()
data = data.drop('Unnamed: 0', 1)

In [43]:
data.head()

,StationID,2017030821,2017030822,2017030823,2017030900,2017030901,2017030902,2017030903,2017030904,2017030905
0,72,-1,0,-2,0,0,3,-1,-2,-3
1,79,0,0,0,0,-1,0,0,0,-3
2,82,1,0,0,0,0,0,0,0,0
3,83,2,2,2,3,-1,-2,-3,0,0
4,116,2,-2,0,-2,-2,3,0,0,-3


In [44]:
data['StationID'] = data['StationID'].astype(str)
data = data.set_index(['StationID'])
data.columns.name = 'Time'
data = data.transpose()
data.head()

StationID,72,79,82,83,116,119,120,127,128,143
Time,,,,,,,,,,
2017030821,-1,0,1,2,2,1,2,-2,1,0
2017030822,0,0,0,2,-2,0,0,-2,-1,0
2017030823,-2,0,0,2,0,0,0,0,0,0
2017030900,0,0,0,3,-2,0,0,0,0,0
2017030901,0,-1,0,-1,-2,0,0,0,-1,0


In [47]:
datehour = list(data.index)
stationid = list(data.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['score']).reset_index()

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878"]
mapper = LinearColorMapper(palette=colors, low=df.score.min(), high=df.score.max())

source = ColumnDataSource(df)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Demo: 10 Citi Bike Docking Station Scores".format(datehour[0], datehour[-1]),
           x_range=datehour, y_range=list(reversed(stationid)),
           x_axis_location="above", plot_width=900, plot_height=400,
           tools=TOOLS, toolbar_location='below')

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="Time", y="StationID", width=1, height=1,
       source=source,
       fill_color={'field': 'score', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

p.select_one(HoverTool).tooltips = [
     ('Date-Hour', '@Time'),
    ('StationID', '@StationID'),
     ('Score', '@score'),
]

show(p)      # show the plot